In [1]:
import pandas as pd
import numpy as np
import os
import heapq
import scipy.sparse as sparse

## V1.0
#### Innerhalb der Klasse.

In [2]:
class Pre_Processer:
    def __init__(self):
        self.contentType = {"Statistic": "0", "Statista-Dossier": "1", "external Report": "2", "Industry Report": "3"}
        
    def read_data_full(self):
        self.data_users = pd.read_csv("../StatistaUsers10000.csv")
        self.data_tracking = pd.read_csv("../StatistaTracking10000.csv")
        
    def read_data_tracking(self):
        self.data_users = pd.read_csv("../StatistaUsers10000.csv")
        self.data_tracking = pd.read_excel("../data_tracking.xlsx")
        self.data_tracking = self.data_tracking.drop("Unnamed: 0", axis=1)
        
    def clean_data(self):        
        data_tracking = self.data_tracking[self.data_tracking.idContent != 0]
        data_tracking["fullId"] = data_tracking.apply(lambda x: (self.contentType[x["contentSubType"]] if x["contentSubType"] in self.contentType else "4") + "_" + str(x["idContent"] ), axis=1)
        data_tracking["date"] = data_tracking.apply(lambda x: pd.to_datetime(f"{x.day}-{x.month}-{x.year}"), axis=1)
        
        self.data_tracking = data_tracking
        self.data_tracking.to_excel(f"../data_tracking.xlsx")
        
    def get_sample(self, fractions, i_frame):        
        return np.array_split(self.data_tracking, fractions)[i_frame]
    
    def create_timesplit(self, date):
        self.training_set = self.data_tracking[self.data_tracking.date < date]
        self.test_set = self.data_tracking[self.data_tracking.date >= date]
        
    def create_matrix_sample(self, fractions = 1, i_frame = 0):
        if fractions != 1:
            if fractions < i_frame:
                i_frame = fractions

            self.data_tracking = self.get_sample(fractions, i_frame)
        
        frame = self.data_tracking[["idUser", "fullId"]]
        frame["views"] = np.ones([len(frame["fullId"]),1])
        frame = frame.groupby(by=["idUser", "fullId"]).sum().reset_index()
        
        user = list(np.sort(frame.idUser.unique()))
        content = list(np.sort(frame.fullId.unique()))
        views = list(frame.views)
        rows = frame.idUser.astype('category', categories = user).cat.codes 
        cols = frame.fullId.astype('category', categories = content).cat.codes 
        
        self.sparsity_ofdata = 1 - (len(views) / (len(content) * len(user)))
                
        self.matrix = sparse.csr_matrix((views, (rows, cols)), shape=(len(user), len(content)))
        size_name = self.get_size(fractions)
        sparse.save_npz(f"../ratings_matrix_{self.matrix_size}_{size_name}", self.matrix, compressed=True)
        
        self.matrix_asFrame = frame
        self.matrix_asFrame.to_excel(f"../data_{self.matrix_size}_{size_name}.xlsx")
        
        self.content_list = content
        
    def create_matrix_timesplit(self):
        training_set = self.training_set
        test_set = self.test_set
        
        # training_set
        frame = self.training_set[["idUser", "fullId"]]
        frame["views"] = np.ones([len(frame["fullId"]),1])
        frame = frame.groupby(by=["idUser", "fullId"]).sum().reset_index()
        
        user = list(np.sort(frame.idUser.unique()))
        content = list(np.sort(frame.fullId.unique()))
        views = list(frame.views)
        rows = frame.idUser.astype('category', categories = user).cat.codes 
        cols = frame.fullId.astype('category', categories = content).cat.codes 
        
        self.sparsity_ofdata = 1 - (len(views) / (len(content) * len(user)))
                
        self.matrix_csr = sparse.csr_matrix((views, (rows, cols)), shape=(len(user), len(content)))
        self.matrix_coo = sparse.coo_matrix((views, (rows, cols)), shape=(len(user), len(content)))
        sparse.save_npz(f"../ratings_matrix_csr", self.matrix_csr, compressed=True)
        sparse.save_npz(f"../ratings_matrix_coo", self.matrix_coo, compressed=True)
        
        self.training_set = frame
        self.training_set.to_excel(f"../training_set.xlsx")
        
        self.content_list = content
        
        # test_set
        frame = self.test_set[["idUser", "fullId"]]
        frame["views"] = np.ones([len(frame["fullId"]),1])
        frame = frame.groupby(by=["idUser", "fullId"]).sum().reset_index()
        
        self.test_set = frame
        self.test_set.to_excel(f"../test_set.xlsx")

In [3]:
Data = Pre_Processer()

In [ ]:
Data.read_data_tracking()
# If fractions is less than 1, the data frame is going to get divided into pieces. Only the i_frame is going to get returned.
Data.clean_data()
Data.create_timesplit("2019-01-01")
Data.create_matrix_timesplit()
#Data.create_matrix(fractions=1, i_frame=47)

In [ ]:
Data.matrix_asFrame.describe()

In [ ]:
Data.matrix_asFrame.tail()

In [ ]:
Data.matrix_asFrame[Data.matrix_asFrame.idUser == 5731].head()

In [ ]:
Data.matrix.toarray()

In [ ]:
int(len(Data.data_tracking.idUser))

In [ ]:
Data.sparsity_ofdata

In [ ]:
1 - Data.sparsity_ofdata

In [10]:
"18.5 Mio. Zeilen"
voller_datensatz = 18498789
"2.69 Mio. Zeilen"
n_2019 = 2688034

In [11]:
n_2019 / voller_datensatz

0.14530864696061996

## V2.0:

In [4]:
Data.read_data_full()

In [5]:
data_tracking = Data.data_tracking[Data.data_tracking.idContent != 0]

In [6]:
data_tracking = data_tracking.iloc[:, 0:6]

In [7]:
data_tracking = data_tracking[data_tracking.year > 2016]

In [8]:
contentType = {"Statistic": "0", "Statista-Dossier": "1", "external Report": "2", "Industry Report": "3"}
def vec_fullId(subType, contentId):
    out = []
    for i in range(len(subType)):
        out.append((contentType[subType[i]] if subType[i] in contentType else "4") + "_" + str(contentId[i]))
        
    return out

In [9]:
data_tracking["date"] = data_tracking['year'].astype(str) + "-" + data_tracking['month'].astype(str) + "-" + data_tracking["day"].astype(str)

In [ ]:
data_tracking["fullId"] = vec_fullId(data_tracking["contentSubType"].values, data_tracking["idContent"].values)

In [ ]:
training_set = data_tracking[data_tracking.date < "2019-04-01"]
test_set = data_tracking[data_tracking.date >= "2019-04-01"]

In [ ]:
# training_set
frame = training_set[["idUser", "fullId"]]
frame["views"] = np.ones([len(frame["fullId"]),1])
frame = frame.groupby(by=["idUser", "fullId"]).sum().reset_index()

user = list(np.sort(frame.idUser.unique()))
content = list(np.sort(frame.fullId.unique()))
views = list(frame.views)
rows = frame.idUser.astype('category', categories = user).cat.codes 
cols = frame.fullId.astype('category', categories = content).cat.codes 

sparsity_ofdata = 1 - (len(views) / (len(content) * len(user)))

matrix_csr = sparse.csr_matrix((views, (rows, cols)), shape=(len(user), len(content)))
matrix_coo = sparse.coo_matrix((views, (rows, cols)), shape=(len(user), len(content)))
sparse.save_npz(f"../ratings_matrix_csr", matrix_csr, compressed=True)
sparse.save_npz(f"../ratings_matrix_coo", matrix_coo, compressed=True)

training_set = frame
#training_set.to_excel(f"../training_set.xlsx")

# test_set
frame = test_set.sort_values(by='date')
frame = frame[["idUser", "fullId"]]
frame["views"] = np.ones([len(frame["fullId"]),1])
frame = frame.groupby(by=["idUser", "fullId"]).sum().reset_index()


test_set = frame
#test_set.to_excel(f"../test_set.xlsx")

np.save("../user_ids", user)
np.save("../content_ids", content)